# AlBi 3
### Übungsblatt 2
*Markus Apel, Mathias Husted*

### Aufgabe 1a)

Es handelt sich um das Protein "Immunoglobulin binding protein"

### Aufgabe 1b, 1c, 1d)



In [ ]:
import numpy as np
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa


class AtomReader:
    def __init__(self, filepath):
        self.parser = PDBParser(QUIET=True)
        self.structure = self.parser.get_structure("structure", filepath)
        self.atom_positions = {}
        # Initialize atom positions
        for model in self.structure:
            for chain in model:
                if chain.id not in self.atom_positions:
                    self.atom_positions[chain.id] = []
                for id, residue in enumerate(chain):
                    if is_aa(residue, standard=True):
                        self.atom_positions[chain.id].append({
                            "id": id,
                            "Residue": residue.get_resname(),
                            "C": None,
                            "O": None,
                            "N": None,
                            "H": None
                        })
                        if "C" in residue:
                            self.atom_positions[chain.id][id]["C"] = residue["C"].get_coord()
                        if "O" in residue:
                            self.atom_positions[chain.id][id]["O"] = residue["O"].get_coord()
                        if "N" in residue:
                            self.atom_positions[chain.id][id]["N"] = residue["N"].get_coord()
                        if "H" in residue:
                            self.atom_positions[chain.id][id]["H"] = residue["H"].get_coord()
    
    def get_sequence(self):
        sequences = {}
        for model in self.structure:
            for chain in model:
                seq = ""
                for residue in chain:
                    if is_aa(residue, standard=True):
                        resname = residue.get_resname()
                        seq += resname + " "
                sequences[chain.id] = seq
        return sequences
    
    def get_positions(self):
        return self.atom_positions
    
    def get_energy(self):
        if self.atom_positions is None:
            return None
        
        h_bond = []

        for chain_id, residues in self.atom_positions.items():
            n_res = len(residues)
            for i, donor in enumerate(residues):
                if donor["N"] is None or donor["H"] is None:
                    continue
                for j, acceptor in enumerate(residues):
                    if acceptor["O"] is None or acceptor["C"] is None:
                        continue
                    if abs(i - j) < 2:
                        continue 

                    r_on = np.linalg.norm(acceptor["O"] - donor["N"])
                    r_ch = np.linalg.norm(acceptor["C"] - donor["H"])
                    r_oh = np.linalg.norm(acceptor["O"] - donor["H"])
                    r_cn = np.linalg.norm(acceptor["C"] - donor["N"])

                    e = 0.084 * ((1/r_on) + (1/r_ch) - (1/r_oh) - (1/r_cn)) * 332
                    h_bond.append(e < -0.5)
        return h_bond
    
    def sec_structure(self, h_bond):
        


reader = AtomReader("1PGA.pdb")

print(reader.get_sequence())

energy = reader.get_energy()

print(len(energy))

{'A': 'MET THR TYR LYS LEU ILE LEU ASN GLY LYS THR LEU LYS GLY GLU THR THR THR GLU ALA VAL ASP ALA ALA THR ALA GLU LYS VAL PHE LYS GLN TYR ALA ASN ASP ASN GLY VAL ASP GLY GLU TRP THR TYR ASP ASP ALA THR LYS THR PHE THR VAL THR GLU '}
2916
